# Chapter 3 Finding Null Space

In [107]:
# numerical and scientific computing libraries  
import numpy as np 
import scipy as sp

# plotting libraries
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [108]:
# for pretty printing
np.set_printoptions(4, linewidth=100, suppress=True)

In [109]:
# create a random matrix of size m x n with the rank <= k <= min(m, n).
def create_random_matrix(m: int, n: int, k: int) -> np.ndarray:
    if k > min(m, n):
        raise ValueError("k must be less than or equal to min(n, m)")
    A = np.random.randn(m, k)
    B = np.random.randn(k, n)
    return A@B

We consider a matrix of size $10 \times 11$ of rank 7, which is non-square and not of full rank.

In [111]:
m=10
n=11
k=7
# generate an mxn matrix of rank at most k
A = create_random_matrix(m, n, k)

To find a null space of a matrix $A$, we apply $LU$-decomposition to $A$. Since the null spaces of $A$ and $U$ coincide, we find a null space of the upper triangular matrix $U$. We characterize the pivot entries by searching the first non-zero element in each row of $U$, and then remaining variables are free ones.

In [ ]:
pivots = []
# LU-decomposition
P, L, U = sp.linalg.lu(A)
print(U)
# identify rank of U
rank_u = np.linalg.matrix_rank(U)
# find pivot entries
for i in range(rank_u):
    non_zero_indices = np.nonzero(U[i,:])
    pivots.append(non_zero_indices[0][0])
print('Pivot variables =')
print(np.int64(pivots))
# find column indices of free variables
free_variables = np.sort(list(set(range(n))- set(pivots)))
print("Free variables =")
print(free_variables)

[[ 3.6313  7.1449 -7.7587 -0.566  -0.6122 -0.5223 -5.0766  3.4328  1.3346  2.4913 -0.1902]
 [ 0.      4.0841 -4.4225 -3.4757 -2.9088  2.5954 -3.1707  2.515  -1.183   0.3487 -2.7929]
 [ 0.      0.     -7.318   3.084   2.0057 -1.1078 -4.4102 -5.7378  0.7763  0.4202 -0.1391]
 [ 0.      0.      0.     -4.4966 -7.3575  4.4918  2.0461 -0.9951  0.6493 -0.8687 -0.396 ]
 [ 0.      0.      0.      0.     -3.0551  0.5613 -2.693   1.1948 -1.4009  0.2045  3.4397]
 [ 0.      0.      0.      0.      0.     -3.1493 -3.585   0.3819  0.7576  0.0848  0.5403]
 [ 0.      0.      0.      0.      0.      0.      4.0348  0.6037 -1.9955  0.1144 -0.7839]
 [ 0.      0.      0.      0.      0.      0.      0.     -0.      0.     -0.     -0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      0.      0.      0.    ]]
Pivot variables =
[0 1 2 3 4 5 6]
Free variables =
[ 7  8  9 10]


Collect pivot columns of U into **U_pivot**, and columns for free variables into **U_free**. Then solve $U_{pivot} X = -U_{free}$ where $X$ is an $n \times (n- Rank A)$ matrix solution of the equation.

In [114]:
# Collect pivot columns and free variables
U_pivot = U[:rank_u, pivots]
print('pivot columns of U = ')
print(U_pivot)
U_free = U[:rank_u, free_variables]
print('free variable columns of U = ')
print(U_free)
# Solve for the pivot part of a basis of the null space of A
X_pivot = np.linalg.solve(U_pivot,-U_free)
print('pivot part of X = ')
print(X_pivot)
# Combine pivot part and free variable part
X = np.eye(n)[:,free_variables]
X[pivots,:] = X_pivot

print('X = ')
print(X)

np.allclose(A@X, 0)

pivot columns of U = 
[[ 3.6313  7.1449 -7.7587 -0.566  -0.6122 -0.5223 -5.0766]
 [ 0.      4.0841 -4.4225 -3.4757 -2.9088  2.5954 -3.1707]
 [ 0.      0.     -7.318   3.084   2.0057 -1.1078 -4.4102]
 [ 0.      0.      0.     -4.4966 -7.3575  4.4918  2.0461]
 [ 0.      0.      0.      0.     -3.0551  0.5613 -2.693 ]
 [ 0.      0.      0.      0.      0.     -3.1493 -3.585 ]
 [ 0.      0.      0.      0.      0.      0.      4.0348]]
free variable columns of U = 
[[ 3.4328  1.3346  2.4913 -0.1902]
 [ 2.515  -1.183   0.3487 -2.7929]
 [-5.7378  0.7763  0.4202 -0.1391]
 [-0.9951  0.6493 -0.8687 -0.396 ]
 [ 1.1948 -1.4009  0.2045  3.4397]
 [ 0.3819  0.7576  0.0848  0.5403]
 [ 0.6037 -1.9955  0.1144 -0.7839]]
pivot part of X = 
[[ 1.4054 -2.7155 -0.0802 -0.1322]
 [-2.3656  1.8633 -0.3823 -0.4059]
 [-0.9768  0.273  -0.0391 -0.5422]
 [-0.9414  1.6079 -0.3152 -1.5963]
 [ 0.5765 -0.9537  0.1028  0.9455]
 [ 0.2916 -0.3224  0.0592 -0.0496]
 [-0.1496  0.4946 -0.0283  0.1943]]
X = 
[[ 1.4054 -2.7155 

True

Check the linear independence of columns of matrix $X$ since, after row shuffling, a submatrix of $X$ is an identity matrix.